In [ ]:
%matplotlib inline
import pickle
import matplotlib.pyplot as plt
import numpy as np

# from decoding_toolbox_py.Helper_funcs import DecToolbox as dt
# from decoding_toolbox_py.Helper_funcs.base_stats import corr_linear_circular
# from decoding_toolbox_py.Helper_funcs.classifier_funcs import scorer_angle, AngularRegression

import DecToolbox as dt
from base_stats import corr_linear_circular
from classifier_funcs import scorer_angle, AngularClassifier



# import DecToolbox as dt # pip install dectool 

from mne import read_epochs
from mne.decoding import SlidingEstimator,GeneralizingEstimator

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.svm import LinearSVR, SVC
# from base_stats import corr_linear_circular
from sklearn import preprocessing
# from classifier_funcs import AngularRegression, scorer_angle
import time as tm
from sklearn.model_selection import KFold


time = gvaverage.times
n_time = len(time)
n_time = 2
#%%capture --no-display
clf = SVC(kernel='linear')
clf = SVC(kernel='linear', probability=True)
a = tm.time() 
nfold = 5 # folds for crossvalidations

nSubj = np.size(subjs_list)
scorer_deci_subjs = np.zeros((nSubj,n_time))
scorer_deci_fold_subjs = np.zeros((nfold,n_time))
lab = preprocessing.LabelEncoder()

for ival, isubj in enumerate(subjs_list):
    print(ival, isubj)
    Y = all_rawdata[ival]['epoch_dat'] 
    all_rawdata[ival]['metadata']['condv'] = 'C'
    all_rawdata[ival]['metadata'].loc[all_rawdata[ival]['metadata']['cond']== 1, 'condv'] =  'D'
    dv  = np.array(all_rawdata[ival]['metadata'].condv)
    dv = lab.fit_transform(dv)
    
    #dv = lab.fit_transform(dv)
   # binned_dv = np.digitize(dv, bins = np.array([ -0.3,0,0.3 ]) ) # bins collapsing degrees by similarity

    FoldsIdx = dt.CreateFolds(dv, Y, nfold)
    
    for ifold, fold in enumerate(FoldsIdx):
        train_ix = fold['train_index']
        test_ix = fold['test_index']
        print('hi')
        for it in range(n_time):
        # for it in range(10):
            Ytime = Y[:,:,it]
            scaler = StandardScaler().fit(Ytime)
            Ytime = scaler.transform(Ytime)
            
            clf.fit( Ytime[train_ix,:], dv[train_ix])
            preds = clf.predict(X = Ytime[test_ix])
            scorer_deci_fold_subjs[ifold, it] = prob_scorer(dv[test_ix],preds)
            
    scorer_deci_subjs[ival, : ] = np.mean(scorer_deci_fold_subjs, axis = 0)
b = tm.time()                          